In [1]:
from itertools import product
from tqdm import tqdm
from collections import defaultdict

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from transformers import AutoModelForCausalLM


from experiments import prune_experiment, hyperparameter_experiments
from utils import get_data, load_gpt2_tl

torch.set_grad_enabled(False)

device = "cuda" if torch.cuda.is_available() else "cpu"

%load_ext autoreload
%autoreload 2

/home/jgcarrasco/.virtualenvs/mech_interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# acronyms -> 250
# ioi -> 150
# greater-than -> 250
n_patching = 250
n_val = 250
task = "greater-than"

data = get_data(n_patching=n_patching, n_val=n_val, task=task)

patching_tokens = data["patching_tokens"].cuda()
patching_answer_tokens = data["patching_answer_tokens"].cuda() 
patching_logits = data["patching_logits"]
patching_cache = data["patching_cache"]

val_tokens = data["val_tokens"].cuda()
val_answer_tokens = data["val_answer_tokens"].cuda()
val_logits = data["val_logits"]
val_cache = data["val_cache"]

gt_circuit = data["gt_circuit"]

Loaded pretrained model gpt2-small into HookedTransformer


In [2]:
hyperparameter_experiments("acronym")

Loaded pretrained model gpt2-small into HookedTransformer


UnboundLocalError: cannot access local variable 'patching_tokens' where it is not associated with a value